<a href="https://colab.research.google.com/github/MustafaKhan670093/Machine-Learning-Playbook/blob/master/Converting_Trained_Neural_Networks_To_Different_Formats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Converting Trained NNs To Different Formats**

A lot of RND work is done using Pytorch. However, deploying models on edge devices usually requires a conversion to ONNX or to (Tensorflow and then) Tensorflow Lite. This notebook covers this conversion process.

### **Install Dependencies**

Run the following cell and then restart the runtime so that you can use these libraries.

In [ ]:
!pip install torchvision
!pip install onnx
!pip install  onnx-tf==1.5.0
!pip install tensorflow-addons
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

### **Let's Train A Simple MNIST Classifier**

At the end of this section, a network that can classify MNIST digits has been trained and saved in a `model.pt` file.

In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
num_workers = 0
batch_size = 20
valid_size = 0.2

transform = transforms.ToTensor()
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [4]:
n_epochs = 50
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'mnist.pth')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.267271 	Validation Loss: 0.342553
Validation loss decreased (inf --> 0.342553).  Saving model ...
Epoch: 2 	Training Loss: 0.502955 	Validation Loss: 0.207912
Validation loss decreased (0.342553 --> 0.207912).  Saving model ...
Epoch: 3 	Training Loss: 0.387287 	Validation Loss: 0.157670
Validation loss decreased (0.207912 --> 0.157670).  Saving model ...
Epoch: 4 	Training Loss: 0.330789 	Validation Loss: 0.132818
Validation loss decreased (0.157670 --> 0.132818).  Saving model ...
Epoch: 5 	Training Loss: 0.297623 	Validation Loss: 0.121562
Validation loss decreased (0.132818 --> 0.121562).  Saving model ...
Epoch: 6 	Training Loss: 0.264539 	Validation Loss: 0.104081
Validation loss decreased (0.121562 --> 0.104081).  Saving model ...
Epoch: 7 	Training Loss: 0.252966 	Validation Loss: 0.099216
Validation loss decreased (0.104081 --> 0.099216).  Saving model ...
Epoch: 8 	Training Loss: 0.236357 	Validation Loss: 0.101093
Epoch: 9 	Training Loss: 0.218656 

### **Load The Saved Pytorch Model And Export It As An ONNX File**

In [5]:
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

#Load Saved Pytorch Model
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

#Export to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28)) 
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")

### **Load The ONNX File And Import It With Tensorflow**

In [6]:
#Load ONNX file
model = onnx.load('mnist.onnx')

#Import ONNX model with Tensorflow
tf_model = prepare(model)

### **Test The Tensorflow Model To Check It Works**

In [18]:
import tensorflow as tf
import numpy as np
from IPython.display import display
from PIL import Image
print('Test Image')
img = Image.open('/content/img1.png').resize((28, 28)).convert('L')
display(img)
output = tf_model.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as a', np.argmax(output))

Test Image


The digit is classified as a 5


### **Save The Tensorflow Model**

In [8]:
tf_model.export_graph('mnist.pb')

INFO:tensorflow:Assets written to: mnist.pb/assets


INFO:tensorflow:Assets written to: mnist.pb/assets


In [19]:
import os

#Helper functions to calculate model file sizes
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

TENSORFLOW_MODEL_NAME = '/content/mnist.pb/saved_model.pb'      
convert_bytes(get_file_size(TENSORFLOW_MODEL_NAME), "KB")

File size: 112.769 Kilobytes


### **Convert Tensorflow Model To Tensorflow Lite**

In [20]:
#Make a converter object from the saved tensorflow file
converter = tf.lite.TFLiteConverter.from_saved_model('mnist.pb')

#Tell converter which type of optimization techniques to use
converter.optimizations = [tf.lite.Optimize.DEFAULT]

#To view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional

#Convert the model 
tf_lite_model = converter.convert()

#Save the converted model 
open('mnist.tflite', 'wb').write(tf_lite_model)

TF_LITE_MODEL_FILE_NAME = 'mnist.tflite'
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 27.406 Kilobytes


### **Test If Tensorflow Lite Model Works**

In [21]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# If you have a folder of images you want to test, change 1 to the number of images
interpreter.resize_tensor_input(input_details[0]['index'], (28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (1, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

#Loading image from earlier
image = np.array(img, dtype=np.float32)

Input Shape: [28 28]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>


In [22]:
test_imgs_numpy = np.array(image, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)
display(img)
print("The digit is classified as a " + str(prediction_classes[0]))


Prediction results shape: (1, 10)


The digit is classified as a 5


Tensorflow Lite is able to be run on devices such as a phone or a Raspberry Pi (with a Coral USB Accelerator).